In [1]:
import cv2
import numpy as np

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
# config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.8
session = InteractiveSession(config=config)

from gen_net import GEN

Using TensorFlow backend.


In [2]:
fname = 'UBMk30rjy0o.mp4'
path = 'data/val/' + fname

In [3]:
cap = cv2.VideoCapture(path)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 動画の画面横幅
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 動画の画面縦幅
fcnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # 総フレーム数
fps = cap.get(cv2.CAP_PROP_FPS)


In [4]:
r = 3
input_dir = 'data/train/low/'
tgt_dir = 'data/train/high/'

input_shape = (int(h/r), int(w/r), 3)
tgt_shape = (h, w, 3)
model_path = 'models/20200514_generator.h5'

gen = GEN(model_path, input_shape, tgt_shape)

model = gen.generator

/home/keras/gen_net.py:84: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="generator", inputs=Tensor("in..., outputs=Tensor("ou...)`
  self.shared_generator = Network(input=layers[0], output=layers[-1], name='generator')


In [5]:
pix = 0
for l in model.layers:
    s = l.output_shape
    pix += s[1]*s[2]*s[3]

In [6]:
pix * 32 / 1024 / 1024 / 1024 / 8

2.9920578002929688

In [ ]:
output_path = 'test.avi'
fourcc =cv2.VideoWriter_fourcc(*'XVID')
vw = cv2.VideoWriter(output_path, fourcc, fps, (w, h), True)
batch = 3
low_list = []

while(cap.isOpened()):
    ret, img = cap.read()
    if ret != True:
        break
    ret, frame = cap.read()
    low_frame = (cv2.resize(frame , (int(w/r), int(h/r))) - 127.5) / 127.5
    low_list.append(low_frame)
    if len(low_list) == batch:
        pred = model.predict(np.array(low_list))
        imgs =  ((pred - pred.min()) / (pred.max() - pred.min()) * 255).astype('u1')
        for img in imgs:
            vw.write(img)
        low_list = []

In [ ]:
cap.release()
cv2.destroyAllWindows()

# cv2.imwrite('vtest.jpg', img)

In [11]:
cv2.imwrite('test.jpg', imgs[0])

True

In [16]:
img_rev = ((img - img.min()) / (img.max() - img.min()) * 255).astype('u1')

In [18]:
cv2.imwrite('test.jpg', img_rev)

True